In [1]:
import cv2
import numpy as np
import os
import json

In [2]:
sift = cv2.xfeatures2d.SIFT_create()

In [3]:
FLANN_INDEX_KDTREE = 0
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
search_params = dict(checks=50)

flann = cv2.FlannBasedMatcher(index_params, search_params)

In [8]:
uncropped_kpd_cache = {}
print("Building cache...")
files = os.listdir("./prod/sources")
files = [(subdir, os.listdir(f"./prod/sources/{subdir}")) for subdir in files]
files = [f"./prod/sources/{subdir}/{f}" for (subdir, l) in files for f in l]
for i, file in enumerate(files):
    print(f"Computing {file} ({i + 1} / {len(files)}, {int(((i + 1) / len(files)) * 100)}%)")
    uncropped = cv2.imread(file)
    if uncropped is None:
        print(f"Error reading {file}. Skipping.")
        continue
    uncropped_keypoints, uncropped_descriptors = sift.detectAndCompute(uncropped, None)
    uncropped_kpd_cache[file] = (uncropped_keypoints, uncropped_descriptors)

Building cache...
Computing ./prod/sources/00/000248a8761165836471f8f604b16915.jpg (1 / 1976, 0%)
Computing ./prod/sources/00/00186914861e802262b9eefa8a793e85.png (2 / 1976, 0%)
Computing ./prod/sources/00/0018819862ce2e522d8c2deac428cf9f.jpg (3 / 1976, 0%)
Computing ./prod/sources/00/0034e57faddc2d96bee60d297670acf5.jpg (4 / 1976, 0%)
Computing ./prod/sources/00/004a77b6e9bf9099f4f334a12e567a19.png (5 / 1976, 0%)
Computing ./prod/sources/00/0072b09dff59dc4491336b8e4ea82e70.png (6 / 1976, 0%)
Computing ./prod/sources/00/00a834590d744f892272f9ebfdef22c4.jpg (7 / 1976, 0%)
Computing ./prod/sources/01/011463679d067f8645a44b8c18f32229.jpg (8 / 1976, 0%)
Computing ./prod/sources/01/01573ed600b6774cf04fa5b8bd5a3e2c.png (9 / 1976, 0%)
Computing ./prod/sources/01/017eccc7cf51d2044ac7b56db8a92a8d.png (10 / 1976, 0%)
Computing ./prod/sources/01/01972bc6b71821c8a16214a94a4e1994.jpg (11 / 1976, 0%)
Computing ./prod/sources/01/01bf45fa3ea2fbf170bd75ee95b30083.png (12 / 1976, 0%)
Computing ./prod/so

In [9]:
from keypoint_serialization import *

In [10]:
def serialize_kpd(i):
    (kps, d) = i
    skps = [serialize_kp(kp) for kp in kps]
    return (skps, d.tolist())

In [19]:
for filename, kpd in uncropped_kpd_cache.items():
    print(filename)
    (subdir, filename) = os.path.split(filename)
    (subdir, prefix) = os.path.split(subdir)
    subdir = f"./prod/kpd_cache/{prefix}"
    if not os.path.isdir(subdir):
        os.mkdir(subdir)
    with open(subdir + "/" + filename + ".json", "w") as f:
        json.dump(serialize_kpd(kpd), f)

./prod/sources/00/000248a8761165836471f8f604b16915.jpg
./prod/sources/00/00186914861e802262b9eefa8a793e85.png
./prod/sources/00/0018819862ce2e522d8c2deac428cf9f.jpg
./prod/sources/00/0034e57faddc2d96bee60d297670acf5.jpg
./prod/sources/00/004a77b6e9bf9099f4f334a12e567a19.png
./prod/sources/00/0072b09dff59dc4491336b8e4ea82e70.png
./prod/sources/00/00a834590d744f892272f9ebfdef22c4.jpg
./prod/sources/01/011463679d067f8645a44b8c18f32229.jpg
./prod/sources/01/01573ed600b6774cf04fa5b8bd5a3e2c.png
./prod/sources/01/017eccc7cf51d2044ac7b56db8a92a8d.png
./prod/sources/01/01972bc6b71821c8a16214a94a4e1994.jpg
./prod/sources/01/01bf45fa3ea2fbf170bd75ee95b30083.png
./prod/sources/01/01f98e8e278fd3155c40faa3668bdfa6.png
./prod/sources/01/01fa6bdfc1baca4a27a6572ac4111f9e.jpg
./prod/sources/02/02648bdec3afb5584c414be882788c32.jpg
./prod/sources/02/0269f46ad61da1e85032eb1c5e5088a4.jpg
./prod/sources/02/027143a24e7f4b7e45dbf5f4a87e2897.png
./prod/sources/02/028e281b84215642204a284fc549452d.jpg
./prod/sou

AttributeError: 'NoneType' object has no attribute 'tolist'